# Image Warping Lab

## Part 1: Interpolation

Since we are going to be warping images, we need to make sure we have a good interpolation function to deal with inexact pixel locations.

In the cell below, we want to scale an image by a factor of 2.30 in both x and y. **Write a function called "interpolate" that performs bilinear interpolation.** This function should take in the image and a location (x,y) of interest. It then uses the nearby pixels to interpolate and return an RGB value.

**Note:** For this lab, we will keep images as floats to simplify the interpolation. Thus, output your images as values between 0-1, not 0-255.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#Your bilinear interpolation function
def interpolate(image: np.ndarray, x: float, y: float) -> float | int:
    
    h, w, _ = image.shape

    # get neighboring points
    # clip to w/in bounds if necessary
    x0 = max(int(np.floor(x)), 0)
    y0 = max(int(np.floor(y)), 0)
    x1 = min(x0 + 1, w - 1) 
    y1 = min(y0 + 1, h - 1)

    # distances/weights between TL pixel and orginal pixel
    xw = x - x0
    yw = y - y0

    # neighboring pixel values
    p00 = image[y0, x0].astype(np.float32) # TL
    p01 = image[y0, x1].astype(np.float32) # TR
    p10 = image[y1, x0].astype(np.float32) # BL
    p11 = image[y1, x1].astype(np.float32) # BR

    return ( 
        p00 * (1 - xw) * (1 - yw) + 
        p01 * xw * (1 - yw) + 
        p10 * (1 - xw) * yw + 
        p11 * xw * yw
    )

filename = "test.png"
im = plt.imread(filename)

plt.imshow(im,vmin=0)
plt.show()

print(np.amax(im)) #Image values should be between 0-1, not 0-255

h, w, _ = im.shape

result = np.zeros((int(2.3 * h), (int(2.3 * w)), 3), dtype = "float32")

rows, cols, _ = result.shape

# Map each new point back to the original image
for i in range(rows):
    for j in range(cols):
        result[i,j] = interpolate(im, j / 2.3, i / 2.3)
 

plt.imshow(result,vmin=0)
plt.show()

## Part 2: Compose (i.e. Backwards Mapping)

Now that we have a interpolation function, we need a function that can compose a source image onto a target image given a transform. In Lab 3, we used Pillow's built-in compose function. You will now implement this yourself using backwards mapping. **This function should also call your interpolate function.**

To implement this function,
- Invert the given transformation. (This is really easy using the NumPy *linalg* library)
- For each pixel in the target image
  - Determine the x and y coordinates of the pixel
  - Homogenize the coordinate
  - Multiply it by the inverse transform
  - Perform the homogenous divide on the new point
  - If the x,y coordinate is within the boundaries of the source image, grab the interpolated value from source image. Otherwise, keep the original pixel in the target image.

For this example, a simple rotation transformation will be given. The source and target image will be the same size, which means part of your rotated image will be cut off on the corners. If done correctly, you should see the parrot rotated onto a purple background. You can assume that all pixels need to be backward mapped. Any pixel/point that maps to a point outside the image boundaries should be skipped. **Don't forget the homogenous divide.**

In [ ]:
def compose(source: np.ndarray, target: np.ndarray, transform: np.ndarray) -> np.ndarray:
    
    # dimension grabbing
    h, w, _ = source.shape
    target_h, target_w, _ = target.shape

    result = np.copy(target)
    
    # invert transformation matrix for backward mapping
    transform_inv = np.linalg.inv(transform)

    for y in range(target_h):
        for x in range(target_w):

            # homogenize
            # aka add one dimension
            xy_target_homogenized = np.array([x, y, 1]) # (x, y) to (x, y, 1)

            # backward mapping
            xy_backward_map = transform_inv @ xy_target_homogenized.T # convert to column vector by using "T"
            xy_backward_map = np.array(xy_backward_map).flatten() # convert to 1d array

            # homogenous divide to get source coordinates
            x_source = xy_backward_map[0] / xy_backward_map[2]
            y_source  = xy_backward_map[1] / xy_backward_map[2]

            # just another classic boundary check
            if 0 <= x_source < w and 0 <= y_source < h: # we can siimply discard anything not in bounds

                # NOTE FOR FUTURE LOGAN
                ''' 
                    We multiplied the inverse of the transformation matrix with a homogenized x and y from the target image.
                    
                    That process is a "map" to find the exact x and y coords from a source.
                    
                        Now that we've found the position of WHERE the source image's x and y are...

                        ...we interpolate them to grab an estimated pixel color from that location.
                    
                    Once we have the pixel, we set the current x and y coordinate of the target to that pixel.
                '''
                result[y, x] = interpolate(source, x_source, y_source)
    
    return result


from math import sin, cos, pi

filename = "test.png"
im = plt.imread(filename)
canvas = np.zeros_like(im) + np.array([[[ 80/255, 45/255, 127/255 ]]]) #ECU Purple

transform = np.matrix([[cos(45 * pi/180), -sin(45 * pi/180), im.shape[1]/2],
                       [sin(45 * pi/180), cos(45 * pi/180), -im.shape[0]/5],
                       [0, 0, 1]])

result = compose(im, canvas, transform)

plt.imshow(result, vmin = 0)
plt.show()


## Part 3: Perspective Warp

Now that we have the two specific functions that we need, let's start looking at some more interesting image warping. In class, we discussed how we can use homographies to perform perspective warps. In this lab, we have provided the homography generating code for you. 

In [3]:
class Point():
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def __str__(self):
        return "({},{})".format(self.x,self.y)
    
    def __repr__(self):
        return self.__str__()


def getHomography(s0, s1, s2, s3, t0, t1, t2, t3):

    x0s = s0.x
    y0s = s0.y
    x0t = t0.x
    y0t = t0.y

    x1s = s1.x
    y1s = s1.y
    x1t = t1.x
    y1t = t1.y

    x2s = s2.x
    y2s = s2.y
    x2t = t2.x
    y2t = t2.y

    x3s = s3.x
    y3s = s3.y
    x3t = t3.x
    y3t = t3.y

    #Solve for the homography matrix
    A = np.matrix([
            [x0s, y0s, 1, 0, 0, 0, -x0t*x0s, -x0t*y0s],
            [0, 0, 0, x0s, y0s, 1, -y0t*x0s, -y0t*y0s],
            [x1s, y1s, 1, 0, 0, 0, -x1t*x1s, -x1t*y1s],
            [0, 0, 0, x1s, y1s, 1, -y1t*x1s, -y1t*y1s],
            [x2s, y2s, 1, 0, 0, 0, -x2t*x2s, -x2t*y2s],
            [0, 0, 0, x2s, y2s, 1, -y2t*x2s, -y2t*y2s],
            [x3s, y3s, 1, 0, 0, 0, -x3t*x3s, -x3t*y3s],
            [0, 0, 0, x3s, y3s, 1, -y3t*x3s, -y3t*y3s]
        ])

    b = np.matrix([
            [x0t],
            [y0t],
            [x1t],
            [y1t],
            [x2t],
            [y2t],
            [x3t],
            [y3t]
        ])

    #The homorgraphy solutions a-h
    solutions = np.linalg.solve(A,b)

    solutions = np.append(solutions,[[1.0]], axis=0)

    #Reshape the homography into the appropriate 3x3 matrix
    homography = np.reshape(solutions, (3,3))
    
    return homography

We want to be able to get a new image into the TV set in the image shown below. 

To place the bird correctly inside the TV, do the following:

- Determine the homography that maps from the bird image to the TV. Use the `getHomography` function and the provided points.

- For each pixel/point within the target image, backmap those points onto the bird image. Use your `compose` function and make sure it correctly rejects points that do not land within the image boundary

Write a general perspective warp function that completes these steps. **Hint: If your compose/backmap function is written correctly, this should only be 2-3 lines of code.**

In [ ]:
def perspectiveWarp(source_im ,target_im, s0, s1, s2, s3, t0, t1, t2, t3):
    '''
        NOTE Dear future Logan,

            Remember, the homography matrix (in this context) is just the transformation matrix.

            If you have all 8 points -- 4 from the source, and 4 from the destination -- the homography 
            matrix describes the relationship of change between the two images.

            In this example, we just took the corner coordinates of the source image, and then the 
            four coordinates of the destination to map the original image to the new location.
    '''
    return compose(source_im, (result := np.copy(target_im)), getHomography(s0, s1, s2, s3, t0, t1, t2, t3))
    

filename = "test.png"
im = plt.imread(filename)

h,w,_ = im.shape
        
s0 = Point(0,0)
s1 = Point(w-1,0)
s2 = Point(w-1,h-1)
s3 = Point(0,h-1)

t0 = Point(245,152)
t1 = Point(349,150)
t2 = Point(349,253)
t3 = Point(246,261)

tv = plt.imread('tv.png')
plt.imshow(tv,vmin=0)
plt.show()

result = perspectiveWarp(im,tv,s0,s1,s2,s3,t0,t1,t2,t3)

plt.imshow(result,vmin=0)
plt.show()


# Part 4: Speed Up

We have a complete perspective warping algorithm now, but you may have noticed that it runs a little slowly. This is because it checks every pixel in the target image to see if it needs to be backmapped. A much faster solution is create a min-max bounding box around the warping points and only checking pixels within that bounding box. **See the bounding box example image provided with the lab.** You may find `np.amax()` and `np.amin()` to be helpful.

For this last part of the lab, rewrite your perspective warp and compose functions as needed so that they only check pixels in the relevant bounding box in the image. Compare the run times between the previous approach and your new method.

In [ ]:
def composeFast(source, target, transform, box = None):   
    
    # dimension grabbing
    h, w, _ = source.shape

    result = np.copy(target)
    
    # invert transformation matrix for backward mapping
    transform_inv = np.linalg.inv(transform)

    for y in range(box[0].y, box[1].y + 1):
        for x in range(box[0].x, box[1].x + 1):

            # homogenize
            # aka add one dimension
            xy_target_homogenized = np.array([x, y, 1]) # (x, y) to (x, y, 1)

            # backward mapping
            xy_backward_map = transform_inv @ xy_target_homogenized.T # convert to column vector by using "T"
            xy_backward_map = np.array(xy_backward_map).flatten() # convert to 1d array

            # homogenous divide to get source coordinates
            x_source = xy_backward_map[0] / xy_backward_map[2]
            y_source  = xy_backward_map[1] / xy_backward_map[2]

            # just another classic boundary check
            if 0 <= x_source < w and 0 <= y_source < h: # we can siimply discard anything not in bounds
                result[y, x] = interpolate(source, x_source, y_source)
    
    return result

def perspectiveWarpFast(source_im,target_im,s0,s1,s2,s3,t0,t1,t2,t3):

    start = Point(np.amin([t0.x, t1.x, t2.x, t3.x]),
                  np.amin([t0.y, t1.y, t2.y, t3.y]))
    
    end = Point(np.amax([t0.x, t1.x, t2.x, t3.x]), 
                np.amax([t0.y, t1.y, t2.y, t3.y]))
    
    return composeFast(source_im, (result := np.copy(target_im)), getHomography(s0, s1, s2, s3, t0, t1, t2, t3), (start, end))
    

from time import time
    
museum = plt.imread('museum.png')
plt.imshow(museum,vmin=0)
plt.show()

filename = "test.png"
im = plt.imread(filename)

h,w,_ = im.shape
        
s0 = Point(0,0)
s1 = Point(w-1,0)
s2 = Point(w-1,h-1)
s3 = Point(0,h-1)

t0 = Point(268,230)
t1 = Point(349,249)
t2 = Point(347,361)
t3 = Point(267,363)

#Points for nearest canvas
#t0 = Point(0,170)
#t1 = Point(235,222)
#t2 = Point(236,365)
#t3 = Point(0,368)


start = time()
result1 = perspectiveWarp(im,museum,s0,s1,s2,s3,t0,t1,t2,t3)
print("Slow Method:",time()-start)

plt.imshow(result1,vmin=0)
plt.show()

start = time()
result2 = perspectiveWarpFast(im,museum,s0,s1,s2,s3,t0,t1,t2,t3)
print("Fast Method:",time()-start)

#plt.imsave("Output.png",result2)
plt.imshow(result2,vmin=0)
plt.show()

# Part 5: QR Code Reader

We now have all the basic building blocks to make our own QR code reader. You will implement one for the last part of the lab. We will focus primarily the computer vision side of QR codes, which involves detecting, aligning, and discretizing QR codes so that they can be read properly.

### Understanding QR Codes

QR codes (short for Quick Response Codes) were invented in 1994, by the DENSO Corporation. These codes store data in two dimensions in the form of an array of contrasting regions. The information density of a QR code is much higher than a vanilla barcode; depending on the format used and the resolution of reader, over a thousand bytes can be encoded in a region the size of a postage stamp.

QR codes use a Reed–Solomon error correction based technology to help recover from errors in reading (for instance, caused by a smudge, badly printed code or other deformity).

Any QR code can be broken into the following sections:

<div style="width:60%;text-align:center;"><img src="qr-code-images/formatting.jpg" width=100%></div>

On three corners of a QR code are square blocks that the reader uses to coarsely identify and then align the code. These position markers always have a black/white/black/white/black ratio of 1:1:3:1:1, no matter the angle they are approached from.

<div style="width:30%;text-align:center;"><img src="qr-code-images/finder_pattern.jpg" width=100%></div>

The fourth point is called the alignment point and has a slightly different pattern and location. This allows the QR code to be properly oriented. Some smaller QR codes do not have this alignment block, so they approximate this location using an affine transform of the three given points.

Once these four points are identified, the image can be aligned using a homography and warping. The number of rows and columns in the QR code is determined by the timing information which alternates from black and white in both the vertical and horizontal direction. 

Once the image is aligned and the size determined, the QR code is discretized, undergoes an Xor with a particular mask given the format information, and read bit for bit in the following order:

<div style="width:60%;text-align:center;"><img src="qr-code-images/decoding.jpg" width=100%></div>

For more details about the decoding process, see the [DataGenetics Wounded QR Codes Blog](http://datagenetics.com/blog/november12013/index.html).

### Implementation

You will be implementing a QR code reader for 29x29 QR codes. The very first step of identifying the three corners using the 1:1:3:1:1 ratio has been implemented for you. For simplicity, the code does not return a fourth alignment point, but rather returns an axis-aligned point as if a corner was present, as shown in the figure. 

<div style="width:40%;text-align:center;"><img src="qr-code-images/keypoints.png" width=100%></div>

The very last step of taking a NumPy array of 1's and 0's and decoding it using the QR code standard has also been implemented for you. Your task is to implement everything inbetween.

To implement the QR code reader for a given input image:
- Feed the image into the provided `getCornerPoints` function to the get the x,y coordinates of the four corners.
- Use your `perspectiveWarp` to map the four points to an axis-aligned square centered on a 29x29 blank image (sometimes called a buffer or canvas).
- Discretize/threshold the image into True/False values.
- Feed the 29x29 grid into the provided `decode` function. A string will be returned with the QR code URL data.

**Remember that the locations of corners are not on the edge, but 4 pixels in on the 29x29 grid.** Once you have implemented the code, run it on the test cases below.

The decoder function depends on packages called `requests` and `BeautifulSoup` to run an online API call. You may need to `pip install` these packages if the decoder doesn't run properly.


In [6]:
from qr import getCornerPoints, decode

def check_timing(image: np.ndarray, d: int) -> bool:

    # start / stop indices for timing rows and columns
    y = 7 - 1 # finder pattern - spacing
    x = 7 + 1 # finder pattern + spacing
    length = d - 7 - 1 # dimensions - pattern - spacing

    timing_row = image[ y, x : length ]
    timing_col = image[ x : length, y ]

    # timing check
    # NOTE white = True, black = False
    return (np.all(timing_row[0::2] == False) and 
            np.all(timing_row[1::2] == True) and
            np.all(timing_col[0::2] == False) and
            np.all(timing_col[1::2] == True)) 


def build_qr_grid(d: int):
    # dynamically build a 'd x d' canvas
    # finder patterns are universal amongst all QR codes
    return Point(3, 3), Point(d - 4, 3), Point(3, d - 4), Point(d - 4, d - 4)

def scanQRcode(image: np.ndarray) -> np.ndarray:

    # RGBA -> RGB
    image = image[..., :3] if image.shape[2] == 4 else image

    # normalize
    # ...just in case
    image = image.astype(np.float32) / 255.0 if image.max() > 1.0 else image.astype(np.float32)
    
    # homography coords
    s0, s1, s2, s3 = getCornerPoints(image)
    
    # loops through grids '21 x 21' - '41 x 41'
    # stops iterating and returns QR code that passes timing check
    for d in range(21, 42, 4):

        # other homography coords
        t0, t1, t2, t3 = build_qr_grid(d) 
        target = np.zeros(shape = (d, d, 3), dtype = np.float32) # I don't want to rework my other functions, so we're initializing this baby with 3 color channels for absolutely no reason
    
        # warp image -> gray image -> binarize image
        warped = perspectiveWarp(image, target, s0, s1, s2, s3, t0, t1, t2, t3)
        grayed = warped @ [0.2989, 0.5870, 0.1140]
        binarized = grayed > .6
        
        if check_timing(binarized, d):
            plt.imshow(binarized, cmap = 'gray')
            plt.title(f'QR Code {d}x{d}')
            plt.show()

            return decode(binarized)
        
    raise ValueError("something's broken in the loop/check_timing.")

In [ ]:
image1 = plt.imread("qr-code-images/test1.png")
plt.imshow(image1)
plt.title(f'Original QR Code')
plt.show()
code1 = scanQRcode(image1)
print(code1)

In [ ]:
image2 = plt.imread("qr-code-images/test2.png")
plt.imshow(image2);plt.title(f'Original QR Code');plt.show()
code2 = scanQRcode(image2)
print(code2)

### Extra Credit: Any Size QR Code Reader (10 pts)

The QR code reader you implemented will work for 29x29 QR codes, but QR codes can be many sizes. All QR codes have an odd number of bits per row and column and go up by 4, starting at size 21 (i.e. 21x21, 25x25, 29x29, ...). For 5 points extra credit, implement a QR code reader that works for any size from 21x21 up to 41x41.

To check if a QR code matches a given size, discretize the QR code asumming a 21x21 size. Then, determine if the timing information alternates in the appropriate manner (see the Understanding QR Codes for more information). If the timing information is valid, then you can assume that the QR code is the given size. If not, move up to the next size (25x25) and repeat.

You can check if your any-size QR code reader is working by running the following test case.

In [ ]:
image_ec = plt.imread("qr-code-images/extra_credit.png")
plt.imshow(image_ec);plt.title(f'Original QR Code');plt.show()
code_ec = scanQRcode(image_ec)
print(code_ec)